In [210]:
import random
import itertools
import matplotlib.pyplot as plt
import networkx as nx

In [211]:
num_columns = 4
grid_size = 30
min_room_size = 3

roomspace = grid_size - (num_columns + 1)
def all_less_then(comb,n):
    for i in comb:
        if i<n:
            return False
    return True

def get_divisions(n, k):
    divisions = []
    combs = itertools.product(range(1, n), repeat=k)
    dic={}
    for comb in combs:
        if sum(comb) == n and all_less_then(comb,min_room_size):
            dic["".join(map(str,sorted(comb)))]=comb
    return list(dic.values())

divisions = get_divisions(roomspace, num_columns)


horizontal_split = list(random.choice(divisions))
vertical_splits = [list(random.choice(divisions)) for i in horizontal_split]
random.shuffle(horizontal_split)
for split in vertical_splits:
    random.shuffle(split)

horizontal_data = []
x_accum = 1
for part in horizontal_split:
    horizontal_data.append((x_accum,part))
    x_accum += part+1

vertical_data = []
for split in vertical_splits:
    y_accum = 1
    data = []
    for part in split:
        data.append((y_accum,part))
        y_accum += part+1
    vertical_data.append(data)
divisions = get_divisions(roomspace, num_columns)

In [212]:
num_rooms = 5

horizontal_split = list(random.choice(divisions))
vertical_splits = [list(random.choice(divisions)) for i in horizontal_split]
random.shuffle(horizontal_split)
for split in vertical_splits:
    random.shuffle(split)

horizontal_data = []
x_accum = 1
for part in horizontal_split:
    horizontal_data.append((x_accum,part))
    x_accum += part+1

vertical_data = []
for split in vertical_splits:
    y_accum = 1
    data = []
    for part in split:
        data.append((y_accum,part))
        y_accum += part+1
    vertical_data.append(data)


chosen_cells = 0
rooms_ixs = []

while True:
    x = random.choice(range(len(horizontal_split)))
    y = random.choice(range(len(vertical_splits[x])))
    ix = (x,y)
    if ix not in rooms_ixs:
        rooms_ixs.append(ix)
    if len(rooms_ixs) == num_rooms:
        break

chosen_rooms = []
for ix_x,ix_y in rooms_ixs:
    x,w = horizontal_data[ix_x]
    y,h = vertical_data[ix_x][ix_y]
    chosen_rooms.append((y,x,h,w))

def get_points(part):
    points = []
    y = part[0]
    x = part[1]
    for i in range(part[2]):
        for j in range(part[3]):
            points.append((y+i,x+j))
    return points
            
    
walkable_points = []
for w in chosen_rooms:
    walkable_points += get_points(w)
walkable_points = set(walkable_points)

def create_string(walkable):
    string = ""
    for i in range(grid_size):
        for j in range(grid_size):
            if (i,j) in walkable:
                string += " "
            else: string += "#"
        string += "\n"
    return string

print(create_string(walkable_points))

##############################
##########################   #
##########################   #
##########################   #
##########################   #
###################      #   #
###################      #####
###################      #####
###################      #####
###################      #####
###################      #####
##############################
##############################
##############################
##########################   #
##########################   #
#############     ########   #
#############     ########   #
#############     ########   #
#############     ########   #
#############     ########   #
#############     ########   #
##########################   #
#############     ########   #
#############     ############
#############     ############
#############     ############
#############     ############
#############     ############
##############################



In [213]:
def toG(y,x):
    return y-1,x-1
def toL(p):
    y,x = p
    return y+1,x+1
def get_room_center(room):
    y,x,h,w=room
    cy = y + (h//2)
    cx = x + (w//2)
    return (cy,cx)

In [214]:
import numpy as np
from scipy.spatial.distance import cdist

def get_distance(r1,r2):
    A = np.array(get_points(r1))
    B = np.array(get_points(r2))
    distances = cdist(A, B)
    min_distance = np.min(distances)
    return min_distance

distances = {}
for i,r1 in enumerate(chosen_rooms):
    for j,r2 in enumerate(chosen_rooms):
        if i<j:
            distances[(i,j)]=get_distance(r1,r2)


In [215]:
sorted_pairs=list(sorted(map(lambda x:(x[0],x[1]),distances.items()),key=lambda x:x[1]))

In [216]:
import networkx as nx
G=nx.grid_graph(dim=[grid_size-2,grid_size-2])

for e in G.edges(data=True):
    e[2]['weigth']=1
for n,d in G.nodes(data=True):
    d['taken']=False
for y,x in walkable_points:
    G.nodes[(y-1,x-1)]['taken']=True

def set_weights(G,taken,weigth):
    for node in taken:
        for k,v in G[node].items(): #edges connected to the node
            if G.nodes[k]['taken']==True: #ignore if the node on the other side is already taken
                  continue
            else:
                v['weigth']+=weigth
    return G
rooms_points =  map(lambda x:x[0],filter(lambda d: d[1]['taken'] == True, G.nodes(data=True)))
G = set_weights(G,rooms_points,100)

In [217]:
paths = {}
for pair,_ in sorted_pairs:
    r1 = chosen_rooms[pair[0]]
    r2 = chosen_rooms[pair[1]]
    c1=get_room_center(r1)
    c2=get_room_center(r2)
    path=nx.shortest_path(G, source=c1, target=c2, weight='weigth')
    paths[pair]=path
    for y,x in path:
        G.nodes[(y,x)]['taken']=True
    G=set_weights(G,path,50)

In [218]:
paths

{(0, 1): [(19, 15),
  (20, 15),
  (21, 15),
  (22, 15),
  (23, 15),
  (24, 15),
  (25, 15),
  (26, 15)],
 (3, 4): [(3, 27),
  (3, 26),
  (3, 25),
  (4, 25),
  (4, 24),
  (4, 23),
  (4, 22),
  (5, 22),
  (6, 22),
  (7, 22),
  (8, 22)],
 (2, 4): [(19, 27),
  (18, 27),
  (17, 27),
  (16, 27),
  (15, 27),
  (14, 27),
  (14, 26),
  (14, 25),
  (14, 24),
  (14, 23),
  (13, 23),
  (12, 23),
  (11, 23),
  (10, 23),
  (9, 23),
  (9, 22),
  (8, 22)],
 (0, 4): [(19, 15),
  (18, 15),
  (17, 15),
  (16, 15),
  (15, 15),
  (14, 15),
  (13, 15),
  (12, 15),
  (11, 15),
  (10, 15),
  (9, 15),
  (9, 16),
  (9, 17),
  (9, 18),
  (8, 18),
  (8, 19),
  (8, 20),
  (8, 21),
  (8, 22)],
 (0, 2): [(19, 15),
  (19, 16),
  (19, 17),
  (19, 18),
  (19, 19),
  (19, 20),
  (19, 21),
  (19, 22),
  (19, 23),
  (19, 24),
  (19, 25),
  (19, 26),
  (19, 27)],
 (1, 2): [(26, 15),
  (25, 15),
  (24, 15),
  (23, 15),
  (22, 15),
  (22, 16),
  (22, 17),
  (22, 18),
  (22, 19),
  (22, 20),
  (22, 21),
  (22, 22),
  (22, 23)

In [219]:
ix = -1
walkable_points1 = set(list(walkable_points) + list(map(toL,paths[sorted_pairs[ix][0]])))
print(create_string(walkable_points))

##############################
##########################   #
##########################   #
##########################   #
##########################   #
###################      #   #
###################      #####
###################      #####
###################      #####
###################      #####
###################      #####
##############################
##############################
##############################
##########################   #
##########################   #
#############     ########   #
#############     ########   #
#############     ########   #
#############     ########   #
#############     ########   #
#############     ########   #
##########################   #
#############     ########   #
#############     ############
#############     ############
#############     ############
#############     ############
#############     ############
##############################



In [ ]:
def get_room_subgraph(graph, room, room_id):
    y, x, h, w = room
    surrounding_nodes = [(y - 1 + i, x - 1 + j) for i in range(h + 2) for j in range(w + 2) if i in [0, h + 1] or j in [0, w + 1]]
    ret = []
    for node in surrounding_nodes:
        x, y = node
        neighbors = [(x + dx, y + dy) for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]]
        for neighbor in neighbors:
            if neighbor in surrounding_nodes:
                ret.append((node, neighbor))
    print(ret)

    return ret


In [ ]:
get_room_subgraph(G,(1,1,2,2),1)